# Tools
1. Custom Fucntion
2. Built In
3. 3rd Party -> langchain tools using in Autogen

# Custom Tool

In [66]:
import os
from dotenv import load_dotenv
load_dotenv()
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.tools import FunctionTool

api_key = os.getenv("OPEN_ROUTER_API_KEY")

In [67]:
model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
    model="openai/gpt-4o-mini",    
    model_info={
        "family": "openai",
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "structured_output": True
    },
)

In [68]:
def reverse_string(text: str) -> str:
    """
    Reverse te given string
    input: str
    output: str
    The reverse string is returned
    """
    return text[::-1]
reverse_tool =  FunctionTool(reverse_string, description="A tool to Reverse a string")


In [69]:
agent = AssistantAgent(
    name="ReverseStringAgent", 
    model_client=model_client,
    system_message="You are a helpful assistant where you reverse the string given. Give the summary",
    tools = [reverse_tool],
    reflect_on_tool_use= True
)

In python script:

async def main():
    result = await agent.run(task="reverse the string 'hello'")

In [70]:
result = await agent.run(task="reverse the string 'hello world'")
print(result)

messages=[TextMessage(id='f3820969-0ab7-431c-8dba-5781e6190985', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 13, 17, 20, 23, 799772, tzinfo=datetime.timezone.utc), content="reverse the string 'hello world'", type='TextMessage'), ToolCallRequestEvent(id='b4f02290-58a9-448f-9430-bbc4abe09fc8', source='ReverseStringAgent', models_usage=RequestUsage(prompt_tokens=74, completion_tokens=15), metadata={}, created_at=datetime.datetime(2026, 2, 13, 17, 20, 25, 461686, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_RlIUwdiRYz4WkEdIWL9p7gmV', arguments='{"text":"hello world"}', name='reverse_string')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='025a0aa5-06de-4ce4-a8fc-8f6a0290472e', source='ReverseStringAgent', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 13, 17, 20, 25, 470305, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='dlrow olleh', name='reverse_string', call_id='call_Rl

messages=[
    
    TextMessage(id='a70da480-9191-40eb-bc62-36d5153727d1', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 13, 16, 30, 43, 124012, tzinfo=datetime.timezone.utc), content="reverse the string 'hello world'", type='TextMessage'), 
    
    ToolCallRequestEvent(id='1d6d7357-413e-4bcf-9c33-a0c9226da876', source='ReverseStringAgent', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=15), metadata={}, created_at=datetime.datetime(2026, 2, 13, 16, 30, 44, 548390, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_RtZs5PD8nkFQfaATEOFOlGIW', arguments='{"text":"hello world"}', name='reverse_string')], type='ToolCallRequestEvent'), 
    
    ToolCallExecutionEvent(id='c1b62b6b-4912-492c-90f0-d91efbe571a7', source='ReverseStringAgent', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 13, 16, 30, 44, 551188, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='dlrow olleh', name='reverse_string', call_id='call_RtZs5PD8nkFQfaATEOFOlGIW', is_error=False)], type='ToolCallExecutionEvent'), 
    
    ToolCallSummaryMessage(id='96fa525c-6394-4882-826f-a77fb43f09f3', source='ReverseStringAgent', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 13, 16, 30, 44, 551188, tzinfo=datetime.timezone.utc), content='dlrow olleh', type='ToolCallSummaryMessage', tool_calls=[FunctionCall(id='call_RtZs5PD8nkFQfaATEOFOlGIW', arguments='{"text":"hello world"}', name='reverse_string')], results=[FunctionExecutionResult(content='dlrow olleh', name='reverse_string', call_id='call_RtZs5PD8nkFQfaATEOFOlGIW', is_error=False)])] stop_reason=Non

In [71]:
result.messages[-1].content

"The reversed string is 'dlrow olleh'."

# Built in Tool

In [82]:
from autogen_ext.tools.http import HttpTool

In [83]:
schema = {
    "type": "object",
    "properties":{
        "fact": {
            "type": "string",
            "description": "A fact about the cat"
        },
        "length":{
            "type": "integer",
            "description": "The length of the cat in centimeters"
        }
    },
    "required":["fact","length"]
}

In [84]:
http_tool = HttpTool(
    name="cat_fact",
    description="get a random cat fact",
    scheme="https",
    host="catfact.ninja",
    port=443,
    path="/fact",
    method="GET",
    json_schema=schema,
    return_type="json",
)

In [85]:
agent = AssistantAgent(
    name="assistant",
    model_client = model_client,
    system_message="You are a helpful assistant that can use the HTTP tool to make requests to external services. give it in summary.",
    tools = [http_tool],
    reflect_on_tool_use = True,
)
result = await agent.run(task="get a random cat fact")
print(result)

messages=[TextMessage(id='c8eed741-2b58-40d7-8013-29ccc798a74d', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 13, 17, 22, 19, 146940, tzinfo=datetime.timezone.utc), content='get a random cat fact', type='TextMessage'), ToolCallRequestEvent(id='3613cab5-62db-439f-a4d5-8267f73d861c', source='assistant', models_usage=RequestUsage(prompt_tokens=97, completion_tokens=34), metadata={}, created_at=datetime.datetime(2026, 2, 13, 17, 22, 21, 402161, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_VXIWYCA13Tk9ieN30kcuLY0e', arguments='{"fact":"Cats have five toes on their front paws, but only four toes on their back paws.","length":25}', name='cat_fact')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='8fb4c2c3-d4be-4d5d-9935-5d4967ec4890', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 13, 17, 22, 22, 729552, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='{\'fact\

In [86]:
result.messages[-1].content

"Here's a random cat fact: Jaguars are the only big cats that don't roar."